# Từ những thông tin như: Hướng gió, độ ẩm, nhiệt độ, áp suất môi trường,... mà chúng ta có được, có thể kết luận vào ngày hôm đó có mưa hay không?

 Trong cuộc sống thì chúng ta có rất nhiều việc bị phụ thuộc vào những cơn mưa, chẳng hạn như việc tưới cây, nếu trời mưa thì chúng ta không phải tưới cây mà thay vào đó có thể pha 1 ly cà phê, mở một bản nhạc và vừa uống cà phê, vừa ngắm mưa. Thật thú vị đúng không nào?

 Cũng có thể trong những công việc khác, người ta thu thập được các dữ liệu về độ ẩm, nhiệt độ, áp suất nhưng vô tình không có được dữ liệu về việc trời hôm đó có mưa hay không. Việc giải quyết được bài toán này có thể giúp ích rất nhiều trong chuyện đó.
 
 Chính vì lý do đó, nhóm em quyết định sẽ chọn chủ đề cho đồ án cuối kì là việc đi tìm lời giải cho câu hỏi `Ngày hôm đó có mưa hay không?` từ những thông tin có đã có được.
 

In [40]:
import requests
import json
import csv

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
import sklearn
set_config(display='diagram') # Để trực quan hóa pipeline
pd.options.mode.chained_assignment = None  # default='warn'

# 1. Thu thập dữ liệu
- Việc thu thập dữ liệu trong đồ án này, nhóm em quyết định thu thập dữ liệu thời tiết bằng API....
Link: https://www.worldweatheronline.com/developer/api/docs/historical-weather-api.aspx
- API mất phí không? Đây là một api có tính phí với 15$/1 tháng, nhưng họ cho free trial 60 ngày (với giới hạn 500 request) đầu nên nhóm quyết định lấy dữ liệu vào bài học.
- Dữ liệu được lấy bằng cách request vào api, thông tin request json được ghi ở trong file slide.pdf
- Tập dữ liệu trong notebook là dữ liệu thời tiết trong 1 năm.



In [41]:
# request lấy dữ liệu 1 năm 12x30 ngày
# ngày 1-> 28 mỗi tháng, tại vì em lười handle 
# các cột: Ngày + giờ, tempC, Windspeed(km/h),winddir16Point,weatherCode,humidity,visibility(km),pressure,cloudcover


elements = []
for i in range(1,12):
    month = i
    url = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=d1f163d12c3742b6a0742358210901&q=Ho Chi Minh&format=json&date=2013-'+str(month)+'-01&enddate=2013-'+str(month)+'-28&includelocation=yes'

    print(f'GET {url}')
    r = requests.get(url)
    json_pydata = json.loads(r.text)
    for day in range(0,27):

        for j in range(0,7):
            element = []
            element.append(json_pydata['data']['weather'][day]['date']+' - ' +json_pydata['data']['weather'][day]['hourly'][j]['time'])
            element.append(json_pydata['data']['weather'][day]['hourly'][j]['tempC'])
            element.append(json_pydata['data']['weather'][day]['hourly'][j]['windspeedKmph'])
            element.append(json_pydata['data']['weather'][day]['hourly'][j]['winddir16Point'])
            element.append(json_pydata['data']['weather'][day]['hourly'][j]['weatherCode'])
            element.append(json_pydata['data']['weather'][day]['hourly'][j]['humidity'])
            element.append(json_pydata['data']['weather'][day]['hourly'][j]['visibility'])

            element.append(json_pydata['data']['weather'][i]['hourly'][j]['pressure'])
            element.append(json_pydata['data']['weather'][i]['hourly'][j]['cloudcover'])
            elements.append(element)
       

GET http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=d1f163d12c3742b6a0742358210901&q=Ho Chi Minh&format=json&date=2013-1-01&enddate=2013-1-28&includelocation=yes
GET http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=d1f163d12c3742b6a0742358210901&q=Ho Chi Minh&format=json&date=2013-2-01&enddate=2013-2-28&includelocation=yes
GET http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=d1f163d12c3742b6a0742358210901&q=Ho Chi Minh&format=json&date=2013-3-01&enddate=2013-3-28&includelocation=yes
GET http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=d1f163d12c3742b6a0742358210901&q=Ho Chi Minh&format=json&date=2013-4-01&enddate=2013-4-28&includelocation=yes
GET http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=d1f163d12c3742b6a0742358210901&q=Ho Chi Minh&format=json&date=2013-5-01&enddate=2013-5-28&includelocation=yes
GET http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=d1f163d12c3742b6a0742358

Thông tin api: https://www.worldweatheronline.com/developer/api/docs/historical-weather-api.aspx

In [42]:
cols =['Day + Time','TempC','Windspeed(km/h)','Windir16Point','WeatherCode','Humidity (%)','Visibility (km)','Pressure (mb)','Cloudcover (%)']
data_df = pd.DataFrame(elements, columns=cols)

In [43]:
data_df

,Day + Time,TempC,Windspeed(km/h),Windir16Point,WeatherCode,Humidity (%),Visibility (km),Pressure (mb),Cloudcover (%)
0,2013-01-01 - 0,24,3,NNE,113,78,10,1011,20
1,2013-01-01 - 300,23,4,NNE,113,78,10,1010,22
2,2013-01-01 - 600,24,6,N,113,78,10,1011,21
3,2013-01-01 - 900,28,7,N,113,64,10,1012,16
4,2013-01-01 - 1200,32,3,NNW,113,52,10,1010,14
...,...,...,...,...,...,...,...,...,...
2074,2013-11-27 - 600,26,12,ENE,119,89,10,1009,33
2075,2013-11-27 - 900,30,14,E,113,70,10,1009,20
2076,2013-11-27 - 1200,32,12,ESE,116,59,10,1007,31
2077,2013-11-27 - 1500,30,11,ESE,356,72,7,1006,24


## 2. Khám phá dữ liệu

## Ý nghĩa các cột trong bảng
- `Day + Time`: Ngày và giờ 
- `TempC`: Nhiệt độ của ngày hôm đó tính theo độ C.
- `Windspeed(km/h)`: Tốc độ gió tính theo đơn vị km/h.
- `Windir16Point`: Hướng gió, trên la bàn 16 hướng.
- `WeatherCode`: Mã thời tiết, mã này thể hiện tình trạng thơi tiết trong ngày. Mỗi mã ứng với mức độ thời tiết khác nhau nhưng chúng ta chỉ xét trời mưa hay không nên sẽ có các nhóm `WeatherCode` thể hiện trời mưa. Điều này được xử lý ở bước tiếp theo.
- `Humidity (%)`: Độ ẩm không khí tính bằng đơn vị %.
- `Visibility (km)`: Tầm nhìn xa tính bằng km.
- `Pressure (mb)`: Áp suất không khí tính bằng đơn vị mb.
- `Cloudcover (%)`: Độ che phủ của mây tính bằng đơn vị %.

Như đã nói ở phần trên, `WeatherCode` có giá trị là các giá số thể hiện tình trạng thời tiết nhưng link phía dưới. Vì Vậy cần chuyển giá trị ở cột này về các giá trị 0(trời không mưa) và 1(trời mưa). 

https://gist.github.com/dawnvoh/a1f513123f7b6a42577d

In [44]:
# Những mã weather code của khi có mưa
rain_code = ['389','386','359','356','353','314','311','308','305','302','299','296','293','176']
data_df.loc[~data_df["WeatherCode"].isin(rain_code),"WeatherCode"] = "0"
data_df.loc[data_df["WeatherCode"].isin(rain_code),"WeatherCode"] = "1"
data_df = data_df.rename(columns={"WeatherCode": "Rain Or Not"})

In [45]:
# Các cột của dữ liệu có kiểu dữ liệu gì?
data_df.dtypes

Day + Time         object
TempC              object
Windspeed(km/h)    object
Windir16Point      object
Rain Or Not        object
Humidity (%)       object
Visibility (km)    object
Pressure (mb)      object
Cloudcover (%)     object
dtype: object

 Nhìn vào kết quả trên ta thấy tất cả các cột đều có kiểu dữ liệu  object. Nhưng theo quan sát thực tế từ dataframe, chỉ có cột `Day + Time` và cột `Windir16Point` là có kiểu dữ liệu không phải dạng số(Thực tế thì Windir16Point có kiểu dữ liệu dạng Categorical). Vì vậy chúng ta cần phải chuyển tất cả các cột có dạng số về đúng kiểu dữ liệu của nó.

In [46]:
# Dữ liệu có dòng lặp không?
data_df.index.duplicated().sum()

0

 Ta thấy: Sự có mặt của cột Day + Time có vẻ không đóng góp gì cho kết quả mong đợi mà chỉ gây khó khăn cho việc tính toán nên chúng ta sẽ xóa nó đi

In [47]:
save_day = data_df['Day + Time']
data_df = data_df.drop('Day + Time', axis=1)

Việc tiếp theo là chuyển các cột dữ liệu dạng số về các kiểu dữ liệu số.

In [48]:
def convert_col_dtype(col):
    if col.name not in ['Windir16Point']:
        return pd.to_numeric(col, errors='coerce')
    return col
data_df = data_df.apply(convert_col_dtype)
data_df.dtypes

TempC               int64
Windspeed(km/h)     int64
Windir16Point      object
Rain Or Not         int64
Humidity (%)        int64
Visibility (km)     int64
Pressure (mb)       int64
Cloudcover (%)      int64
dtype: object

### Kế đến, chúng ta sẽ kiểm tra các giá trị median, 25%, 75% của từng cột dữ liệu

In [49]:
def missing_ratio(s):
    return s.isna().mean() * 100
def median(df):
    return df.quantile(0.5)
def lower_quartile(df):
    return df.quantile(0.25)
def upper_quartile(df):
    return df.quantile(0.75)
df = data_df.select_dtypes(exclude='object')
df.agg([missing_ratio, min, lower_quartile, median, upper_quartile, max])

,TempC,Windspeed(km/h),Rain Or Not,Humidity (%),Visibility (km),Pressure (mb),Cloudcover (%)
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,21.0,0.0,0.0,28.0,0.0,1004.0,6.0
lower_quartile,25.0,6.0,0.0,63.5,10.0,1007.0,19.0
median,27.0,8.0,0.0,77.0,10.0,1009.0,29.0
upper_quartile,31.0,11.0,0.0,88.0,10.0,1010.0,45.0
max,38.0,23.0,1.0,98.0,10.0,1014.0,83.0


In [50]:
# Tính tỉ lệ cách giá trị của bộ kết quả Rain Or Not
data_df['Rain Or Not'].value_counts(normalize=True) * 100

0    85.185185
1    14.814815
Name: Rain Or Not, dtype: float64

### Nhận xét về dữ liệu
- Với số lượng dữ liệu như trên, có thể nói đó là con số khá lớn. Như vậy chúng ta có thể đưa ra kết luận có thể tin cậy được.
- Giá trị missing ratio bằng 0 ở tất cả các cột. Điều này là một tín hiệu đáng mừng, báo hiệu việc tiền xử lý dữ liệu của chúng ta sẽ dễ dàng hơn rất nhiều so với việc thiếu nhiều loại dữ liệu ở các cột.
- Dữ liệu của cột `Windir16Point` tuy có không phải dạng số nhưng nó chỉ nhận 16 giá trị(categorical). Vì vậy, chúng ta có thể dùng cách khác để mã hóa chúng thành dạng số. 

In [51]:
# Kiểm tra các thông tin của data khi mà trời mưa
df = data_df[data_df["Rain Or Not"] != 0].select_dtypes(exclude='object')
df.agg([missing_ratio, min, lower_quartile, median, upper_quartile, max])

,TempC,Windspeed(km/h),Rain Or Not,Humidity (%),Visibility (km),Pressure (mb),Cloudcover (%)
missing_ratio,0.0,0.00,0.0,0.00,0.0,0.0,0.0
min,23.0,1.00,1.0,44.00,3.0,1004.0,9.0
lower_quartile,27.0,7.75,1.0,65.00,7.0,1005.0,23.0
median,30.0,10.00,1.0,73.00,9.0,1007.0,29.0
upper_quartile,31.0,13.00,1.0,82.25,10.0,1009.0,45.0
max,36.0,23.00,1.0,98.00,10.0,1012.0,83.0


## 3. Tiền xử lý dữ liệu

Vì bộ dữ liệu này khá tốt nên việc tiền xử lý dữ liệu rất nhẹ nhàng, không cần phải làm gì nhiều vì nó đã quá đầy đủ

 Việc đầu tiên cần làm là tách tập input và output ra. Trong bộ dữ liệu này, output chính là cột `Rain Or Not`, input là cột còn lại. Sau đó, chúng ta phải tách ra hai tập riêng lẻ đó là tập train và tập validation. 

In [52]:
#Split dữ liệu
y_sr = data_df["Rain Or Not"] 
X_df = data_df.drop("Rain Or Not", axis=1)
clone_X= X_df

In [14]:
# Tách tập huấn luyện và tập validation theo tỉ lệ 70%:30%
# train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3, 
#                                                               stratify=y_sr, random_state=0)
train_X_df, X_test, train_y_sr, y_test = train_test_split(X_df, y_sr, test_size=0.2, random_state=1)

train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(train_X_df, train_y_sr, test_size=0.25, random_state=1)

Tiếp theo là việc xử lý cột có dạng Categorical. Ta sẽ sử dụng OneHotEncoder để mã hóa cột "Windir16Point" để phục vụ bước mô hình hóa. Sau khi Onehot trên cột Windir16Point thì chúng ta sử dụng StandardScaler với các cột có dạng số còn lại. Tất cả được nối lại thành một Transformer có tên là preprocess_Transformer. Việc làm này giúp chúng ta dễ dàng tạo mô hình và tránh sai sót xảy ra.

In [15]:
preprocess_Transformer = ColumnTransformer(
    transformers=[
        ('cat',  OneHotEncoder(handle_unknown='ignore'), ['Windir16Point']),
        ('scaler', StandardScaler(), ['TempC','Windspeed(km/h)','Humidity (%)','Visibility (km)','Pressure (mb)','Cloudcover (%)'])
    ])
preprocess_Transformer

ColumnTransformer(transformers=[('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['Windir16Point']),
                                ('scaler', StandardScaler(),
                                 ['TempC', 'Windspeed(km/h)', 'Humidity (%)',
                                  'Visibility (km)', 'Pressure (mb)',
                                  'Cloudcover (%)'])])

In [16]:
# dùng phương thức fit_transform 
preprocess_train_X = preprocess_Transformer.fit_transform(train_X_df)
preprocess_train_X

array([[ 0.        ,  0.        ,  0.        , ..., -0.18530525,
        -1.19271665, -0.4731714 ],
       [ 0.        ,  0.        ,  0.        , ..., -0.18530525,
        -0.29349194, -0.15086   ],
       [ 0.        ,  0.        ,  0.        , ..., -0.18530525,
        -0.29349194, -0.95663849],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.34347198,
         0.15612041,  0.70863706],
       [ 0.        ,  0.        ,  0.        , ...,  0.34347198,
         0.60573276, -1.38638702],
       [ 0.        ,  0.        ,  0.        , ...,  0.34347198,
        -2.09194135, -0.41945283]])

## 4. Mô hình hóa dữ liệu
- Trong bước mô hình hóa dữ liệu này, nhóm mình sẽ sử dụng mô hình Neuron Network để phân lớp. Với bộ tham số của MPLClassifier là (`hidden_layer_sizes=(20), activation='tanh', solver='lbfgs', random_state=0, max_iter=2500`).
- Trước hết, cần phải tạo một pipeline đầy đủ các bước tiền xử lý(đã thực hiện ở trên) và bước phân lớp
- Sau đó, thực hiện thử với siêu tham số alpha để xem xét với alpha nào thì mô hình cho ra kết quả tốt nhất.

In [17]:
mlpc = MLPClassifier(hidden_layer_sizes=(20), activation='tanh', solver='lbfgs', random_state=0, max_iter=2500)
full_pipeline = make_pipeline(preprocess_Transformer, mlpc)


In [18]:
full_pipeline.fit(train_X_df, train_y_sr)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Windir16Point']),
                                                 ('scaler', StandardScaler(),
                                                  ['TempC', 'Windspeed(km/h)',
                                                   'Humidity (%)',
                                                   'Visibility (km)',
                                                   'Pressure (mb)',
                                                   'Cloudcover (%)'])])),
                ('mlpclassifier',
                 MLPClassifier(activation='tanh', hidden_layer_sizes=20,
                               max_iter=2500, random_state=0,
                               solver='lbfgs'))])

In [19]:
alphas = [0.001, 0.01, 0.1,  1, 10,  100, 500, 1000]
train_errs = []
val_errs = []

for alpha in alphas:
    full_pipeline.set_params(mlpclassifier__alpha=alpha)
    full_pipeline.fit(train_X_df, train_y_sr)
    train_err = 100*(1 - full_pipeline.score(train_X_df, train_y_sr))
    train_errs.append(train_err)
    
    val_err = (1 - full_pipeline.score(val_X_df, val_y_sr))*100
    val_errs.append(val_err)


In [20]:
# Tìm ra độ lỗi trên tập validation nhỏ nhất đồng thời tìm alpha tương ứng với độ lỗi đó
best_val_err = min(val_errs)
best_alpha = None
for i in range(0, len(val_errs)):
    if val_errs[i] == best_val_err:
        best_alpha = alphas[i]

In [21]:
#Sau khi tìm được best alpha thì thực hiện lại công việc fit mô hình 
full_pipeline.set_params(mlpclassifier__alpha=best_alpha)
full_pipeline.fit(X_df, y_sr)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Windir16Point']),
                                                 ('scaler', StandardScaler(),
                                                  ['TempC', 'Windspeed(km/h)',
                                                   'Humidity (%)',
                                                   'Visibility (km)',
                                                   'Pressure (mb)',
                                                   'Cloudcover (%)'])])),
                ('mlpclassifier',
                 MLPClassifier(activation='tanh', alpha=1,
                               hidden_layer_sizes=20, max_iter=2500,
                               random_state=0, solver='lbfgs'))])

## 5. Đánh giá mô hình sau khi huấn luyện

In [22]:
#Tính Toán độ lỗi của mô hình trên tập test_df
print("Độ lỗi: ", round(100*(1 - full_pipeline.score(X_test, y_test)), 3),"%")

Độ lỗi:  0.721 %


In [37]:
#Dự đoán trên tập test
pred = full_pipeline.predict(X_test)

- Sau khi đã hoàn thành việc tạo model và chạy trên tập test thì em sẽ tạo ra một bảng mới có chứa những thông tin cơ bản của tập X_test và kết quả dự đoán(Prediction), thực tế(Actual) để so sánh.
- Kết quả ở dưới show ra những kết quả mà khi dự đoán, model đã đoán sai so với thực tế.

In [39]:
compare_result = pd.DataFrame(index = y_test.index)
compare_result["TempC"] = X_test["TempC"]
compare_result["Windspeed(km/h)"] = X_test["Windspeed(km/h)"]
compare_result["Pressure (mb)"] = X_test["Pressure (mb)"]
compare_result["Cloudcover (%)"] = X_test["Cloudcover (%)"]
compare_result["Prediction"] = pred
compare_result["Actual"] = y_test

compare_result[compare_result["Prediction"] != compare_result["Actual"]]

,TempC,Windspeed(km/h),Pressure (mb),Cloudcover (%),Prediction,Actual
1625,24,11,1007,40,0,1
1263,28,15,1010,26,1,0
1440,31,18,1006,20,0,1


# Như vậy, chúng ta đã có thể trả lời được câu hỏi "Ngày hôm đó có mưa hay không?" bằng những cơ sở toán học rồi.

Mình mong đồ án này có thể giúp các bạn có thêm những hiểu biết mới về sự tương quan của các yếu tố thiên nhiên đến việc trời mưa hay không, hoặc ít nhất có thể củng cố kiến thức được học thông qua việc đọc code này.